In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [7]:
import kagglehub

path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

100%|██████████| 612M/612M [04:32<00:00, 2.35MB/s] 

Extracting files...


Path to dataset files: C:\Users\hp\.cache\kagglehub\datasets\meowmeowmeowmeowmeow\gtsrb-german-traffic-sign\versions\1


In [58]:
data = ImageDataGenerator(rescale=1/255.0, validation_split=0.25)
train= data.flow_from_directory(
    path+ "/Train",
    target_size = (64,64),
    batch_size= 32,
    class_mode = "categorical",
    subset="training"
    )
val= data.flow_from_directory(
    path+ "/Train",
    target_size = (64,64),
    batch_size= 32,
    class_mode = "categorical",
    subset = "validation",
    shuffle = False
)

Found 29416 images belonging to 43 classes.
Found 9793 images belonging to 43 classes.


In [59]:
test_csv_path = os.path.join(path, 'Test.csv')

if os.path.exists(test_csv_path):
    test_df = pd.read_csv(test_csv_path)
    
    test_df['ClassId'] = test_df['ClassId'].astype(str)
    
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=path, 
        x_col="Path",  
        y_col="ClassId",
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )
else:
    print("Test.csv not found. Cannot load Test data via dataframe.")

Found 12630 validated image filenames belonging to 43 classes.


In [62]:
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dropout


model =models.Sequential([
    layers.Input(shape=(64,64,3)),
    
    layers.Conv2D(32,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),
    
    layers.Conv2D(64,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),


    layers.Conv2D(128,(3,3), padding = 'same'),
    layers.Activation('elu'),
    layers.MaxPooling2D(2,2),
    Dropout(0.2),

    
    layers.Flatten(),
    layers.Dense(256),
    layers.Activation('elu'),
    layers.Dense(43, activation='softmax')  
])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 43)             │        11,051 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,201,707 (8.40 MB)

 Trainable params: 2,201,707 (8.40 MB)

 Non-trainable params: 0 (0.00 B)

In [63]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [65]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,          
    patience=3,          
    verbose=1,
    min_lr=1e-6
)

In [66]:
trained_model = model.fit(
    train,
    validation_data = val,
    epochs = 20,
    callbacks = [early_stop,lr_scheduler]
)

Epoch 1/20
920/920 ━━━━━━━━━━━━━━━━━━━━ 120s 130ms/step - accuracy: 0.8173 - loss: 0.6663 - val_accuracy: 0.8805 - val_loss: 0.5243 - learning_rate: 0.0010
Epoch 2/20
920/920 ━━━━━━━━━━━━━━━━━━━━ 125s 136ms/step - accuracy: 0.9690 - loss: 0.1127 - val_accuracy: 0.8922 - val_loss: 0.5404 - learning_rate: 0.0010
Epoch 3/20
920/920 ━━━━━━━━━━━━━━━━━━━━ 126s 137ms/step - accuracy: 0.9765 - loss: 0.0908 - val_accuracy: 0.8897 - val_loss: 0.7770 - learning_rate: 0.0010
Epoch 4/20
920/920 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.9796 - loss: 0.0757
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
920/920 ━━━━━━━━━━━━━━━━━━━━ 108s 117ms/step - accuracy: 0.9807 - loss: 0.0735 - val_accuracy: 0.8966 - val_loss: 0.6927 - learning_rate: 0.0010


In [67]:
mapping_match = train.class_indices == test_generator.class_indices

if mapping_match:
    print(" Class mappings align perfectly.")
else:
    print(" WARNING: Class mappings do not match!")
    print("Train mapping:", train.class_indices)
    print("Test mapping:", test_generator.class_indices)

 Class mappings align perfectly.


In [68]:
val_preds = model.predict(test_generator)

val_pred_classes = np.argmax(val_preds, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())


print(classification_report(true_classes, val_pred_classes, target_names=class_labels))

395/395 ━━━━━━━━━━━━━━━━━━━━ 319s 810ms/step
              precision    recall  f1-score   support

           0       0.87      0.43      0.58        60
           1       0.86      0.94      0.90       720
          10       0.96      0.96      0.96       660
          11       0.93      0.90      0.92       420
          12       0.96      0.86      0.91       690
          13       0.95      0.99      0.97       720
          14       1.00      1.00      1.00       270
          15       0.83      1.00      0.91       210
          16       0.67      0.95      0.79       150
          17       1.00      0.81      0.90       360
          18       0.90      0.83      0.86       390
          19       0.60      1.00      0.75        60
           2       0.90      0.90      0.90       750
          20       0.88      0.93      0.91        90
          21       0.80      0.57      0.66        90
          22       0.67      0.99      0.80       120
          23       0.80      0.73   

In [69]:
model.save('traffic_classifier.h5')
print("Model saved successfully.")

Model saved successfully.
